# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testting part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing.names’ for more information on the attributes.

In [2]:
!pip install matplotlib

    100% |████████████████████████████████| 13.2MB 61kB/s eta 0:00:011    95% |██████████████████████████████▊ | 12.7MB 6.4MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 3.9MB/s ta 0:00:011


In [3]:
%matplotlib inline
#import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# %aimport my_module
# my_module.py

/Users/uberwach/anaconda2/envs/python35/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [5]:
!pip install sklearn

    100% |████████████████████████████████| 7.6MB 103kB/s ta 0:00:01    36% |███████████▊                    | 2.8MB 5.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /Users/uberwach/Library/Caches/pip/wheels/d7/db/a3/1b8041ab0be63b5c96c503df8e757cf205c2848cf9ef55f85e
Successfully built sklearn


In [6]:
import sklearn

In [7]:
from sklearn import datasets

In [ ]:
datasets.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
#testfile = urllib.URLopener()
#testfile.retrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names", "housing.names")
df = pd.DataFrame(boston.data)
df.columns = ['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X = boston.data
y = boston.target

In [ ]:
print(boston.DESCR)

In [ ]:
df.head(10)

In [ ]:
df.shape

## Least Squares 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
least_squares = LinearRegression(normalize=True)

In [ ]:
least_squares.fit(X, y)

In [ ]:
!pip install seaborn --proxy=audi.in.vwgroup.prox

In [ ]:
import seaborn as sns

sns.barplot(x=least_squares.coef_, y=df.columns, orient="h")
plt.title("Weights of least squares")

But how good is our model on the data?

In [ ]:
least_squares.score(X, y)

In [ ]:
LinearRegression??

The `score` method always returns the metric the model has been optimized for. The `sklearn.metrics` package contains all kinds of metrics.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_pred = least_squares.predict(X) # computes the predictions
mean_squared_error(y, y_pred)

## An unpleasant surprise

But in reality our model has to form predictions on *unseen* data. Let's model this situation.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, X.shape)

In [ ]:
least_squares.fit(X_train, y_train)

In [ ]:
print("Train score: %.4f" % least_squares.score(X_train, y_train))
print("Test score: %.4f" % least_squares.score(X_test, y_test))

The test score is way worse! This is the effect of overfitting.

*Exercise*: Compute the MSE for the train and test set. How does it compare to the situation before?

In [ ]:
# %load ./solutions/mse.py

## Other models...

Try other models, play with the parameters and try to beat the best score!

*Extra*: Inspect the features not used by the Lasso model and remove them from the feature set. Does this improve the performance of all (others included!) models?

In [ ]:
from sklearn.linear_model import Ridge # L_2
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [ ]:
lasso = Lasso(alpha=0.05, normalize=True)
lasso.fit(X_train, y_train)

In [ ]:
lasso.coef_

In [ ]:
print("Train score: %.4f" % lasso.score(X_train, y_train))
print("Test score: %.4f" % lasso.score(X_test, y_test))

In [ ]:
used_features = lasso.coef_ != 0.0
print(used_features)

In [ ]:
X_reduced = X[:, used_features]

X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_reduced, y, random_state=42)

$\lambda$ must be lowered because there is less noise to learn from.

In [ ]:
lasso = Lasso(alpha=0.01, normalize=True)
lasso.fit(X_train_red, y_train_red)

In [ ]:
print("Train score: %.4f" % lasso.score(X_train_red, y_train_red))
print("Test score: %.4f" % lasso.score(X_test_red, y_test_red))

## Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
clf_pipe = make_pipeline(
    PolynomialFeatures(degree=5),
    ElasticNet(alpha=0.001, l1_ratio=0.3, normalize=True, random_state=42)
)

clf_pipe.fit(X_train, y_train)

In [ ]:
print("Train score: %.4f" % clf_pipe.score(X_train, y_train))
print("Test score: %.4f" % clf_pipe.score(X_test, y_test))

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test, clf_pipe.predict(X_test))

In [ ]:
clf_pipe.steps

In [ ]:
clf_pipe.steps[1][1].coef_.shape